# import

In [7]:
import pandas as pd
from tqdm.notebook import tqdm as tqdm
import time
from graphviz import *
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, fmin,STATUS_OK
import fasttext as ft
from my_library.horse import *

# path

In [2]:
path_ubu = '/home/hipro/デスクトップ/Horse/Data/20_21'
path_mac2 = '/Users/rince/Desktop/Horse/Data/saishin2/'
path_mac = '/Users/rince/Desktop/Horse/Data/saishin/'
path_win = '/Users/Owner/Desktop/program/Horse/Data/saishin/'
path_win2 = '/Users/Owner/Desktop/program/Horse/Data/saishin2/'

# load data

In [3]:
results = load_csv(path_mac+'results.csv')
horse_results = load_csv(path_mac+'horse_results.csv')
peds = load_csv(path_mac+'peds.csv')
# 何回やってもロードすると, nanが出る
peds.fillna('nan',inplace=True)
return_tables = load_csv(path_mac+'return.csv')
return_tables.rename(columns={'0':0,'1':1,'2':2,'3':3},inplace=True)

# race_id 命名規則

race_id 202105040802\
yyyy_pp_xx_xxrr\
y : year\
p : palce\
x : 謎\
r : race番号

In [ ]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 9,
                'learning_rate': 0.07042118817890154,
                'num_iterations': 93,
                'num_leaves': 40
            }

In [4]:
ll = LearnLGBM(peds,results,horse_results)
ll.learn_lgb()


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.070915 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46022
[LightGBM] [Info] Number of data points in the train set: 85884, number of used features: 172


In [ ]:
# ll = LearnLGBM(peds,results,horse_results)
# ll.get_train_data()

val_train_query = ll.x_train.groupby(ll.x_train.index).size()
val_x = ll.x_test.iloc[:-10000]
val_y = ll.y_test.iloc[:-10000]
test_query = val_x.groupby(val_x.index).size()
train = lgb.Dataset(ll.x_train, ll.y_train, group=train_query)
valid = lgb.Dataset(val_x, val_y, reference=train, group=test_query)

# 正解ラベル　変えた

In [ ]:
def split_data(df, test_size=0.2, rank_learning=True):
    """
    データを学習データと, 訓練データに分ける関数
    """
    df_ = df.copy()
    df_['rank'] = df_['rank'].map(lambda x: int((1/x)*10) if x<4 else(0))
        
    sorted_id_list = df_.sort_values("date").index.unique()
    train_id_list = sorted_id_list[: round(len(sorted_id_list) * (1 - test_size))]
    test_id_list = sorted_id_list[round(len(sorted_id_list) * (1 - test_size)) :]
    train = df_.loc[train_id_list]#.drop(['date'], axis=1)
    test = df_.loc[test_id_list]#.drop(['date'], axis=1)
    return train, test

    
train, test = split_data(ll.r.data_c.fillna(0),test_size=0.2,rank_learning=False)
x_train = train.drop(['rank', 'date','単勝'], axis=1)
y_train = train['rank']
x_test = test.drop(['rank', 'date','単勝'], axis=1)
y_test = test['rank']
train_query = x_train.groupby(x_train.index).size()
train = lgb.Dataset(x_train, y_train, group=train_query)
val_y = y_test.iloc[:-10000]
test_query = x_test.groupby(x_test.index).size()
valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

In [ ]:
lgbm_params = {
                'metric': 'ndcg',
                'objective': 'lambdarank',
                'ndcg_eval_at': [1,2,3],
                'boosting_type': 'gbdt',
                'random_state': 777,
                'lambdarank_truncation_level': 10,
                'learning_rate': 0.02273417953255777,
                'num_iterations': 97,
                'num_leaves': 42
            }

{'lambdarank_truncation_level': 10,
 'learning_rate': 0.02273417953255777,
 'num_iterations': 97,
 'num_leaves': 42}

In [ ]:
# {'lambdarank_truncation_level': range(1,50)[33], 'learning_rate': 0.09608503291165846, 'num_iterations': range(50,120)[14], 'num_leaves': range(20,50)[2]}


{'lambdarank_truncation_level':  range(1,50)[9], 'learning_rate': 0.02273417953255777, 'num_iterations': range(50,120)[47], 'num_leaves': range(20,50)[22]}

In [ ]:
ll.learn_lgb2(train,lgbm_params=lgbm_params)

In [ ]:
def score(params):
    print("Training start:")

    N_boost_round = []
    Score = []

    lgb_results={}  #履歴格納用
    # train = lgb.Dataset(x_train, y_train, group=train_query)
    # valid = lgb.Dataset(x_test, y_test, reference=train, group=test_query)

    
    lgb_clf = lgb.train(
        params,
        train,
        num_boost_round=1000,
        valid_sets=valid,
        valid_names=['valid'],
        early_stopping_rounds=20,
        verbose_eval=5,
        evals_result=lgb_results,
    )
#     return lgb_results
    return {'loss': -1.0 * lgb_results['valid']['ndcg@3'][lgb_clf.best_iteration], 'status': STATUS_OK}

def optimize(trials):
#探索スペース
    space = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        # 上位三着を考慮する
        'ndcg_eval_at': [1,2,3],
#         best paramsの返り値は, choiceだとindexか?
        'lambdarank_truncation_level': hp.choice('lambdarank_truncation_level',range(1,50)),
        'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
        # n_estimators == num_iterations
        'num_iterations': hp.choice('num_iterations',range(50,120)),
        'num_leaves': hp.choice('num_leaves',range(20,50)),
        'boosting_type': 'gbdt',
        'random_state': 777
    }

    max_evals = 100      #探索回数(25くらいで十分)
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=max_evals)

    print("best parameters:", best)

In [ ]:
trials = Trials()
optimize(trials)

# Simulate


In [6]:
sl.pred_odds_list

AttributeError: 'RankSimulater' object has no attribute 'pred_odds_list'

In [11]:

start_time = time.time()
for odds in [1.1,2.0,3.0,4.0,5.0,6.0,7.0]:
    print()
    sl = RankSimulater(ll.model)
    print("odds",odds)
    sl.calc_tansho(ll.r.data_c.iloc[-5000:],return_tables,odds=odds)
    print("time", time.time() - start_time)



odds 1.1
not_bet_count 1
---------------------
単勝
的中率 : 79 / 361
的中% : 21.88 %
収支   : -3800.0 円
的中レース : {'01': 8, '02': 8, '03': 8, '04': 7, '05': 7, '06': 6, '07': 8, '08': 7, '09': 2, '10': 6, '11': 6, '12': 6}
time 8.93926191329956

odds 2.0
not_bet_count 45
---------------------
単勝
的中率 : 79 / 317
的中% : 24.92 %
収支   : -2900.0 円
的中レース : {'01': 8, '02': 8, '03': 8, '04': 7, '05': 7, '06': 6, '07': 8, '08': 7, '09': 2, '10': 6, '11': 6, '12': 6}
time 17.911739826202393

odds 3.0
not_bet_count 112
---------------------
単勝
的中率 : 79 / 250
的中% : 31.60 %
収支   : -1600.0 円
的中レース : {'01': 8, '02': 8, '03': 8, '04': 7, '05': 7, '06': 6, '07': 8, '08': 7, '09': 2, '10': 6, '11': 6, '12': 6}
time 27.75297212600708

odds 4.0
not_bet_count 178
---------------------
単勝
的中率 : 79 / 184
的中% : 42.93 %
収支   : 890.0 円
的中レース : {'01': 8, '02': 8, '03': 8, '04': 7, '05': 7, '06': 6, '07': 8, '08': 7, '09': 2, '10': 6, '11': 6, '12': 6}
time 36.24506592750549

odds 5.0
not_bet_count 216
---------------------

In [10]:
sl = RankSimulater(ll.model)
sl.calc_sanrenpuku(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
sl = RankSimulater(ll.model)
sl.calc_sanrentan(ll.r.data_c.iloc[-5000:].fillna(0),return_tables)

In [ ]:
pt = Predictor(peds,results,horse_results,race_id_list)
pt.learn_lgb(lgbm_params=lgbm_params)

# 新潟

In [ ]:
base  = '2022040101'
pt.predict(base+'05')

# 東京

In [ ]:
base  = '2022050205'
pt.predict(base+'05')

# 中京

In [ ]:
base  = '2022070301'
pt.predict(base+'05')

In [ ]:
pt.show_results_today()

# 日付に注意

In [ ]:
date = '2022/12/31'

# race_id_list

In [ ]:


race_id_list = ['2022050205{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022050206{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022070301{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022070302{}'.format(str(i).zfill(2)) for i in range(1,13)]

race_id_list += ['2022040101{}'.format(str(i).zfill(2)) for i in range(1,13)]
race_id_list += ['2022040102{}'.format(str(i).zfill(2)) for i in range(1,13)]



# Results scraping

In [ ]:
# race_id_list = results.index.astype('str')

results_tmp = Results.scrape(race_id_list)

# Horse_results scraping

In [ ]:
horse_id_list = results_tmp['horse_id'].astype(str).unique()
horse_results_tmp = HorseResults.scrape(horse_id_list)
# save_path = '/Users/rince/Desktop/Horse/Data/horse_2020.csv'

# Peds scraping

In [ ]:
peds_2021 = Peds.scrape(horse_id_list)
pe_2021 = Peds(peds_2021)
pe_2021.regularize_peds()
peds_tmp = pe_2021.peds_re

# Return scraping

In [ ]:
returns_tmp = Return.scrape(race_id_list)

# update

In [ ]:
new_results = update_data(load_csv(path_mac+'results.csv'),results_tmp)
new_horse_results = update_data(load_csv(path_mac+'horse_results.csv'),horse_results_tmp)
new_peds = update_data(load_csv(path_mac+'peds.csv'),peds_tmp)
new_return = update_data(load_csv(path_mac+'return.csv'),returns_tmp)

# save

In [ ]:
new_results.to_csv(path_mac+'results.csv')
new_horse_results.to_csv(path_mac+'horse_results.csv')
new_peds.to_csv(path_mac+'peds.csv')
new_return.to_csv(path_mac+'return.csv')

# 重要度

In [ ]:
importances = pd.DataFrame(
{'features' : x_train.columns, 'importances' : lgb_rank.feature_importance()})
print(importances.sort_values('importances', ascending=False)[:30])

# fastText

流れ
1. fasttext用の血統データの学習データを作る (血統の情報のみ, index ヘッダはいらない)
2. fasttext学習
3. 学習モデルを使って, 血統データをベクトル化
4. ベクトル化して r.data_cに concat
5. 学習


教師あり, 教師なしでも生成されるベクトルは等しい

# model_ft 作成

In [ ]:
# 相対パスしかできない
# dim : 出力の次元
# minn : n_gramの最小単位
# maxn : n_gramの最大単位
path_ft = '/Users/rince/Desktop/Horse/code/horse/peds_ft.txt'
# 上書き保存OK
peds.to_csv(path_ft,header=False,index=False,sep=',')
model_ft = ft.train_unsupervised(path_ft,dim=62,minn=2,maxn=14)

model[model.words[1]] と model.get_input_vector(ind=1) は等価

# rank = [int((1/i)*10)] と変換してみてはどうか